## #Note

The reason why I made this file is that the previous one was so comlicated.  
Now, save.jl is able to perform simulations which was individually conducted in analysis.ipynb

2024/1/28


Import libraries

In [2]:
ENV["GKS_ENCODING"]="UTF-8"

using LinearAlgebra
using Statistics
using Plots
using JSON
using HDF5
using StatsPlots

gr(fontfamily="Helvetica")
default(fontfamily="Helvetica")

Define constants

In [3]:
colors = [:orange, :black, :gold, :blue, :red, :brown, :bisque4, :gray, :blue, :black] #惑星の色
AU = 1.495978e8 #AU
G = 6.67430e-20 #gravitational constant (G)

6.6743e-20

Calculate orbit elements below
- periapsis
- apoapsis
- period

In [4]:
function calc_orbit_elems(N, file, tEnd, dt, chunk_size)
    Nt = Int(ceil(tEnd/dt))
    peri = []
    apo = []
    period = []
    bef2 = 0.0
    bef1 = 0.0
    tmp = 0.0

    @time for chunk in 1:Int(Nt/chunk_size)
        tmp = read(file, "/data/$chunk")
        for i in 1:chunk_size
            if bef2 > bef1 < tmp[i] # periapsis
                push!(peri, bef1/AU)
            elseif bef2 < bef1 > tmp[i]
                push!(apo, bef1/AU) # apoapsis
                push!(period, (chunk_size*(chunk-1)+i-1)*dt/86400) # period
            end
            bef2 = bef1
            bef1 = tmp[i]
        end
    end

    if length(peri) < length(apo)
        pop!(apo)
    elseif length(peri) > length(apo)
        pop!(peri)
    end

    return peri, apo, diff(period)
end

calc_orbit_elems (generic function with 1 method)

Load simulation results and run the functions above

In [ ]:
f1 = h5open("./results/dt1hr_tEnd360000d/n_distance_sun_earth_bary.h5", "r")
f2 = h5open("./results/dt1hr_tEnd360000d/n_distance_sun_earth_bary_ex_jupiter.h5", "r")
# f1 and f2 must have the same params as each other's
N = read(f1, "/params/n")
tEnd = read(f1, "/params/tend")
dt = read(f1, "/params/dt")
chunk_size = read(f1, "/params/chunksize")

peri1, apo1, period1 = calc_orbit_elems(N, f1, tEnd, dt, chunk_size)
peri2, apo2, period2 = calc_orbit_elems(N, f2, tEnd, dt, chunk_size)

close(f1)
close(f2)

  0.824599 seconds (34.46 M allocations: 591.886 MiB, 10.53% gc time)
  0.769872 seconds (34.46 M allocations: 591.886 MiB, 5.62% gc time)



Make a boxplot of eccentricity for thesis

In [ ]:
box = boxplot(
    xticks = (1:9, ["全惑星+月", "水星除外", "金星除外", "火星除外", "木星除外", "土星除外", "天王星除外", "海王星除外", "月除外"]),
    widen=false,
    ylims=(0.0162, 0.0172),
    #ylims=(364.9,365.6),
    ylabel=" ",
    legend=false,
    dpi=300,
    rotation=45
)
pls = ["mercury", "venus", "mars", "jupiter", "saturn", "uranus", "neptune"]
filename=""
for i in 1:9
    if i == 1
        filename = "n_distance_sun_earth_bary.h5"
    elseif i == 9
        filename = "n_distance_sun_earth_ex_moon.h5"
    else
        filename = "n_distance_sun_earth_bary_ex_$(pls[i-1]).h5"
    end
    f = h5open("./results/dt1hr_tEnd360000d/$filename", "r")
    N = read(f, "/params/n")
    tEnd = read(f, "/params/tend")
    dt = read(f, "/params/dt")
    chunk_size = read(f, "/params/chunksize")
    peri, apo, period = calc_orbit_elems(N, f, tEnd, dt, chunk_size)
    close(f)
    ec = (apo-peri)./(apo+peri)

    boxplot!(ec)
end
boxplot!(box, xrotation=15)
boxplot!(box, yrotation=0)
boxplot!(box, ylabel="離心率")
savefig(box,"boxplot_eccentricity_all.svg")

  0.745501 seconds (34.46 M allocations: 591.886 MiB, 6.63% gc time)
  0.730580 seconds (34.46 M allocations: 591.886 MiB, 6.15% gc time)
  0.728542 seconds (34.46 M allocations: 591.886 MiB, 5.94% gc time)
  0.731661 seconds (34.46 M allocations: 591.886 MiB, 6.09% gc time)
  0.730151 seconds (34.46 M allocations: 591.886 MiB, 6.21% gc time)
  0.734340 seconds (34.46 M allocations: 591.886 MiB, 6.15% gc time)
  0.756239 seconds (34.46 M allocations: 591.886 MiB, 6.09% gc time)
  0.733973 seconds (34.46 M allocations: 591.886 MiB, 6.38% gc time)
  0.734229 seconds (34.46 M allocations: 591.886 MiB, 6.15% gc time)


Calculate eccentricity from apoapsis and periapsis

In [104]:
ec1 = (apo1-peri1)./(apo1+peri1)
ec2 = (apo2-peri2)./(apo2+peri2)

985-element Vector{Float64}:
 0.016675461233465534
 0.01664607914821124
 0.016659160773596898
 0.016682141383756247
 0.016672737002102058
 0.016662656770515132
 0.016665358818824708
 0.01668311892570372
 0.016682975500004246
 0.01665490439304802
 ⋮
 0.01664004138858623
 0.016630477656293913
 0.016638099159087913
 0.01665183482651181
 0.01664792439143662
 0.01662108679664952
 0.016637228576466212
 0.01665964453325013
 0.016644474299264966

Make a boxplot of eccentricity for thesis

In [ ]:
p_peri = boxplot(
    [[0.0162], ec1, [0.0162], ec2, [0.0162]],
    xticks = (2:2:4, ["全惑星+月", "木星除外"]),
    widen=false,
    ylims=(0.0162, 0.0172),
    ylabel="離心率",
    color=:palegreen3,
    legend=false,
    size=(300,400),
    dpi=300
)

p_period = boxplot(
    [[364.9],period1,[364.9],period2,[364.9]],
    xticks = (2:2:4, ["全惑星+月", "木星除外"]),
    ylims=(364.9,365.6),
    widen=false,
    ylabel="公転周期（日）",
    color=:cadetblue3,
    legend=false,
    size=(300,400),
    dpi=300
)

boxplot!(p_peri, xrotation=15)
boxplot!(p_peri, yrotation=0)
boxplot!(p_period, xrotation=15)
boxplot!(p_period, yrotation=0)

p = plot(p_peri, p_period, layout=(1,2), size=(600,400))

#display(p)
savefig(p, "boxplot_eccentricity_period_jupiter.svg")
#savefig(q, "results/charts/perihe_with_or_without_mars")

"/Users/shim/Desktop/Tanq/Planet/Gravitational_Simulation/amp.svg"

### Stacked Area Chart of Acceleration Proportions

In [102]:
using LinearAlgebra
using Statistics
using Plots
using FFMPEG
using JSON
using HDF5

#=
length:km, mass:kg, time:s, angle:rad(deg in json)
(interval must be a multiple of dt, tEnd must be a multiple of interval)
=#

colors = [:orange, :black, :gold, :blue, :red, :brown, :bisque4, :gray, :blue, :black] #惑星の色
AU = 1.495978e8 #AU
G = 6.67430e-20 #gravitational constant
PLANETS  = ["sun", "mercury", "venus", "earth", "mars", "jupiter", "saturn", "uranus", "neptune", "moon"]
PLANETS_JP  = ["sun", "水星", "金星", "地球", "火星", "木星", "土星", "天王星", "海王星", "moon"]

function accByOthers(N, file, tEnd, dt, mass, title, chunk_size, target)
    planets_fill_color = [:snow3, :goldenrod2, :dodgerblue2, :firebrick, :goldenrod4, :wheat1, :paleturquoise1, :blue4, :gray]

    p = plot(title=title, xlabel="経過時間(時間)", ylabel="加速度の大きさ(km/s^2)", legendfontsize=8, dpi=300)
    Nt = Int(ceil(tEnd/dt))
    chunk = zeros(N,3,chunk_size) #allocate memory in advance
    accs_by_planets = zeros((N, chunk_size))
    accs_sum = zeros(chunk_size)

    used_labels = Set{String}()
    push!(used_labels, PLANETS_JP[target]) 

    for c in 1:Int(Nt/chunk_size)
        chunk = read(file, "/data/$c")
        for i in 1:chunk_size
            accs_sum[i], accs_by_planets[:,i] = getAcc(chunk[:,:,i], mass, N, target)
        end
        for i in 1:N-1
            accs_sum .-= accs_by_planets[i,:]

            current_label = PLANETS_JP[i + 1] # to avoid doubled label
            if current_label in used_labels
                label = "" 
            else
                label = current_label
                push!(used_labels, current_label)
            end

            plot!(p, 1+(c-1)*chunk_size:c*chunk_size, accs_sum, fillrange=0, color=planets_fill_color[i], label=label)
        end
    end
    return p
end

function getAcc(pos, mass , N, target)
    x = pos[target,1]
    y = pos[target,2]
    z = pos[target,3]
    accs_by_planets = zeros(N)
    ax = 0
    ay = 0
    az = 0
    for i in 2:N
        dx = pos[i,1] - x
        dy = pos[i,2] - y
        dz = pos[i,3] - z
        
        inv_r3 = (dx^2 + dy^2 + dz^2 + 1.0e-15).^(-1.5)
        _ax = (dx * inv_r3) * mass[i]
        _ay = (dy * inv_r3) * mass[i]
        _az = (dz * inv_r3) * mass[i]
        ax += _ax
        ay += _ay
        az += _az
        accs_by_planets[i] = G * hypot(_ax,_ay,_az)
    end
    acc_sum = G * hypot(ax,ay,az) # sum of acceleration "vector"
    accs_by_planets = accs_by_planets ./ sum(accs_by_planets) .* acc_sum

    return acc_sum, accs_by_planets
end

function loadMassData(key, epoch)
    file = JSON.parsefile("../Resources/nasa_data.json")
    return file[epoch][key]["mass"]
end

function main()
    # Simulation must include all planets.

    f = h5open("../Gravitational_Simulation/results/dt1hr_tEnd3600d/n_all_ex_moon.h5", "r")
    tEnd = read(f, "/params/tend")
    dt = read(f, "/params/dt")
    chunk_size = read(f, "/params/chunksize")
    N = read(f, "/params/n")
    mass = zeros(N)
    EPOCH = "2024-04-01"
    for i in 1:N
        mass[i] = loadMassData(PLANETS[i], EPOCH)
    end
    p = accByOthers(N, f, tEnd, dt, mass, "", chunk_size, 4)

    #display(p)
    savefig(p, "results/charts/acceleration_to_earth_ex_moon.svg")
    close(f)
end
gr()
main()


In [4]:
colors = [:orange, :black, :gold, :blue, :red, :brown, :bisque4, :gray, :blue, :black] #惑星の色
AU = 1.495978e8 #AU

function plotAllWithChunks(N, file, tEnd, dt, range, title, chunk_size)
    p = scatter([], [], xlims=(-range,range), ylims=(-range,range), title=title, xlabel="x(au)", ylabel="y(au)", aspect_ratio=:equal, legend=false)
    Nt = Int(ceil(tEnd/dt))
    for c in 1:Int(Nt/chunk_size)
        chunk = read(file, "/data/$c")
        for i in 2:N
            X = chunk[i,1,:] ./ AU
            Y = chunk[i,2,:] ./ AU
            plot!(p, X, Y, color=colors[i])
        end
    end
    return p 
end

function main()
    f1 = h5open("./results/dt1hr_tEnd36000d/n_all.h5", "r")
    N = read(f1, "/params/n")
    tEnd = read(f1, "/params/tend")
    dt = read(f1, "/params/dt")
    chunk_size = read(f1, "/params/chunksize")

    @time p1 = plotAllWithChunks(N, f1, tEnd, dt, 5.4, "Planets Orbits", chunk_size)

    close(f1)
    savefig(p1, "orbitno")
end
gr()
main()

  0.934675 seconds (1.70 M allocations: 638.623 MiB, 35.87% gc time, 54.31% compilation time: 65% of which was recompilation)


"/Users/shim/Desktop/Tanq/Planet/Gravitational_Simulation/orbitno.png"